# Transformando leis em dados

Nesse artigo, quero mostrar como podemos importar e criar dataframes a partir de leis federais brasileiras em python utilizando a estrutura do texto legal.

## Analisando leis

Leis são documentos jurídicos e técnicos que normatizam e orientam diferentes aspectos da vida cotidiana. Por diversas razões, estudiosos querem analisar e comparar o conteúdo de diferentes leis. 
<br><br>
Uma maneira de faze-lo é por meio da leitura minuciosa da lei em questão, mas qualquer pessoa que já tenha trabalhado direta ou indiretamente com legislações sabe que nem tudo o que está escrito é relevante em todos os casos. 
<br><br>
Por exemplo, digamos que você more em um condomínio e o síndico deseja proibir que moradores tenham animais de estimação. Para saber se o síndico pode  ou não fazer isso, você deve consultar as leis 4.591/64 (Lei de Condomínios) e 10406/2002 (Novo Código Civil). No entanto, essas duas leis são gigantescas e versam sobre diversas matérias, e  o que você quer saber está em provavelmente um ou dois parágrafos dentro de artigos específicos. 
<br><br>
Desta forma, muitas vezes não queremos estudar uma lei inteira, apenas uma parte da lei, de forma que, quando consideramos em transformar aquelas informações em dados, devemos saber a relevância que cada parte da estrutura legal possui.
<br><br>

## Estrutura (e padrões) das leis:

[Este texto](https://www.politize.com.br/estrutura-das-leis-entenda/) do Guilhermo Glassman para o "Politize-se" explica de forma muito didática o que quer dizer cada parte da estrutura de uma lei. Em resumo:

- **Entidade de origem:** indica se a lei é federal, estadual ou municipal
- **Referência da lei e data de criação**
- **Tipo de lei:** ordinária, complementar, medida provisória, emenda constitucional, decret legislativo e resolucão
- **Ementa:** Resumo do que é tratado na lei
- **Preâmbulo:** Justificativa ou contexto no qual a lei foi criada
- **Conteúdo:**
  * Título
  * Capítulo
  * Artigo:
    * Caput: Cabeça do artigo, texto que vem logo depois de "Artigo 1°", por exemplo, orienta o queas demais subdivisões vão falar
    * Parágrafos ( § ): Destacam aspectos importantes do artigo que não estão expressos no caput
    * Incísos (I, II, ...): Descrevem hipóteses em que a regra é aplicada
    * Alínea (a, b, ...): Subdivisões dos incisos
  
Desta forma, sabemos que se procurarmos palavras-chave em leis, elas terão um peso e uma interpretação de acordo com a parte da estrutura em que ela foi mencionada. 
<br><br>
No entanto, ao observar as leis brasileiras, vemos que nem todos os documentos seguem à risca a estrutura interna de um artigo. Em muitos casos, os artigos seguem do *caput* direto para os *incísos*, enumerando as situações em que se aplica o artigo. Um exemplo da lei 12.527/2011 (Lei de Acesso à Informação):

> Art. 4º Para os efeitos desta Lei, considera-se:

> I - informação: dados, processados ou não, que podem ser utilizados para produção e transmissão de conhecimento, contidos em qualquer meio, suporte ou formato;

> II - documento: unidade de registro de informações, qualquer que seja o suporte ou formato;


Com isso, acho que criar um dataframe a partir de leis que considere apenas os caputs e artigos mais interessante, já que o processo de automatização para achar incisos e alíneas poderia levar a uma interpretação equivocada dos dados.

## Importando e criando o DataFrame

Vou trabalhar aqui com a Lei de Acesso à Informacão (LAI), que consultei inúmeras vezes quando trabalhava na [Transparência Brasil](www.transparencia.org.br) . A Lei 12.527/2011 está disponível [nesse link](http://www.planalto.gov.br/ccivil_03/_ato2011-2014/2011/lei/l12527.htm) . 
<br><br>
Normalmente eu faria um scrap da página do governo com o ``requests.get`` , mas por alguma razão eu obtive diversas vezes um erro de conexão que persistiu mesmo com a instalação de pacotes de segurança. (Acredito que o problema seja do site do Palácio do Planalto, se alguém teve essa dificuldade e encontrou uma solução, entre em contato comigo) <br><br>
Como eu pretendo trabalhar com poucas leis, pude me dar ao luxo de imprimir a página como pdf e importa-la no Python com o ``pdfminer``, que retorna uma grande string contendo toda a lei. <br><br>
Em seguida, utilizei a própria estrutura do texto para fazer o split que cria o dataframe. Os artigos são precedidos de duas quebras de linha ('\n\nArt.') , assim como os parágrafos (\n\n§) , que permitiu a automatização. O resultado que eu buscava era o seguinte: <br><br>

| conteudo | tipo | artigo_referencia | paragrafo_referencia
| ----------- | ----------- | ----------- | ----------- |
| Aplicam-se as disposições desta Lei, no que couber (...) | caput | 1 | NaN |
| A publicidade a que estão submetidas as entidades (...) | parágrafo | 16 | 1 |


Ao aplicar esse código para outras leis, recomendo conferir o número de artigos e parágrados, verificando se artigo ou parágrafo deixou de ser considerado pois a estrutura não estava correta. Ao que tudo indica, as leis são publicadas a partir de um documento de texto que é editado manualmente, o que pode gerar erros (por exemplo, uma quebra de linha ao invés de duas).<br><br>
Abaixo o código final comentado:


In [1]:
# Bibliotecas necessárias:

import pdfminer
from pdfminer.high_level import extract_text
import re
import numpy as np
import pandas as pd

In [12]:
# Transformando a minha lei em uma STRING
lai = extract_text('lai.pdf')

# Evitando espaços duplos, para não haver problema nos splits mais pra frente:
lai = re.sub(r'  ', ' ', lai)

# Criando uma lista com todos os artigos separados:
lai_art = re.split(r'\n\nArt. ', lai)

# Rotina para criar dois dfs: um dos caputs e outro dos parágrafos:
caput = pd.DataFrame({'conteudo': [], 
                   'tipo': [], 
                   'artigo_referencia': [] })

par = pd.DataFrame({'conteudo': [], 
                   'tipo': [], 
                   'artigo_referencia': [],
                   'paragrafo_referencia': []})

for n in range(len(lai_art)):
    i = lai_art[n]
    a = i[0:5]
    a = re.sub(r'[^0-9]+', '', a) 
    if "\n\nParágrafo único" in i:
        i = pd.DataFrame({'conteudo': re.split("\n\nParágrafo único", i)})
        i['paragrafo_referencia'] = 'paragrafo unico'
    else:
        i = pd.DataFrame({'conteudo': re.split("\n\n§", i)})
        i['paragrafo_referencia'] = i['conteudo'].astype(str).str[0:4]
        i['paragrafo_referencia'] = i['paragrafo_referencia'].str.replace(r'[^0-9]+', '')
    i['artigo_referencia'] = a
    c = i[0:1]  # Caputs
    c['tipo'] = 'caput'
    c['paragrafo_referencia'] = np.nan
    p = i[1:]    # Parágrafos
    p['tipo'] = 'paragrafo'
    caput = pd.concat([caput , c], axis = 0, sort=False)
    par = pd.concat([par , p], axis = 0, sort=False)

# Retirando o preâmbulo e outras coisas que vêm antes do conteúdo:
caput = caput[1:] 

#juntando os dfs:
lai_df = pd.concat([par , caput], axis = 0, sort=False)

# retirando quebras de linha dupla, quebras de linha e whitespaces duplicados:
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'\n\n', ' ')
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'\n', ' ')

# Limpando início do conteúdo da lei
lai_df['conteudo'] = lai_df['conteudo'].str.replace(r'^[^a-zA-Z]+', ' ')

# Ressetando o index
lai_df = lai_df.reset_index(drop=True, inplace=False)

# Setando a IDE para exibir todas as linhas do DF
pd.set_option('display.max_rows', None)  

lai_df

<ipython-input-12-ff118cd0eaf6>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['tipo'] = 'caput'
<ipython-input-12-ff118cd0eaf6>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['paragrafo_referencia'] = np.nan
<ipython-input-12-ff118cd0eaf6>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,conteudo,tipo,artigo_referencia,paragrafo_referencia
0,Subordinam-se ao regime desta Lei: I - os órg...,paragrafo,1,paragrafo unico
1,A publicidade a que estão submetidas as entid...,paragrafo,2,paragrafo unico
2,O acesso à informação previsto no caput não c...,paragrafo,7,1
3,Quando não for autorizado acesso integral à i...,paragrafo,7,2
4,O direito de acesso aos documentos ou às info...,paragrafo,7,3
5,A negativa de acesso às informações objeto de...,paragrafo,7,4
6,Informado do extravio da informação solicitad...,paragrafo,7,5
7,Verificada a hipótese prevista no § 5º deste ...,paragrafo,7,6
8,Na divulgação das informações a que se refere...,paragrafo,8,1
9,"Para cumprimento do disposto no caput, os órg...",paragrafo,8,2
